In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import itertools, more_itertools
import swifter
import re

In [2]:
import sys
sys.path.append('..')

from utilities import *

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', 100)

sns.set_context('talk')
sns.set_style('darkgrid', {'patch.edgecolor' : 'none', 'patch.force_edgecolor' : False})

# Load and Organize the Data
The data is in a rather weird format so this will take some work

In [4]:
questions = pd.read_excel('Nature_PhD survey_Anon_v1.xlsx', nrows = 1).transpose()
questions.rename({0 : 'Question Text'}, axis = 'columns', inplace = True)

In [5]:
# print(questions.to_string())

In [6]:
df = pd.read_excel('Nature_PhD survey_Anon_v1.xlsx', skiprows = [1])

In [7]:
df.head()

,ID.format,ID.completed,ID.language,ID.site,ID.date,ID.start,ID.endDate,ID.end,ID.time,Q1,Q1.a,Q2,Q3,Q3.a,Q4,Q5,Q6,Q6.a,Q7,Q7.a,Q8,Q8.a,Q9,Q9.a,Q10,Q10.a,Q11,Q11.a,Q12:1,Q12:2,Q12:3,Q12:4,Q12:5,Q12:6,Q12:7,Q12:8,Q12:9,Q12:10,Q12:11,Q12.a,Q13,Q14,Q14.a,Q15.a,Q15.b,Q15.c,Q15.d,Q15.e,Q15.f,Q15.g,...,Q52:3,Q52:4,Q52:5,Q52:6,Q52:7,Q52:8,Q52.a,Q53:1,Q53:2,Q53:3,Q53:4,Q53:5,Q53:6,Q53:7,Q53.a,Q54:1,Q54:2,Q54:3,Q54:4,Q54:5,Q54.a,Q55,Q56,Q57,Q58:1,Q58:2,Q58:3,Q58:4,Q58:5,Q58:6,Q58:7,Q58:8,Q58:9,Q58:10,Q58:11,Q58:12,Q58.a,Q59:1,Q59:2,Q59:3,Q59:4,Q59:5,Q59.a,Q60,Q61,Q62,Q63,Q64,Q65.a,Q65.b
0,SNAP 2015 SHIFT,completed,Spanish,NaN,06/14/2019,16:46:41,06/14/2019,17:36:11,49.50,Doctorate degree (PhD/DPhil/MD),NaN,NaN,No job I want is available without a PhD,NaN,Yes,North or Central America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,1st,NaN,NaN,7th,3rd,4th,...,NaN,NaN,NaN,NaN,NaN,"Other, please specify",Escuché tutoriales acerca de resilencia y opin...,LinkedIn,NaN,Facebook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, please specify",Trataría de organizar mejor mi tiempo para ten...,La actitud positiva y constructiva no lo es to...,25 - 34,Female (including trans female),NaN,Latino/Hispanic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, to an adult(s)",NaN,NaN,NaN,Felicidades por esta encuesta. Pienso que se h...,Yes,Yes,"Yes, I would like Nature Careers to contact me...","Yes, I would like to join the Shift Learning r...",1,NaN
1,SNAP 2015 SHIFT,completed,English,NaN,06/14/2019,17:10:02,06/14/2019,17:38:25,28.38,Doctorate degree (PhD/DPhil/MD),NaN,NaN,I want to pursue an academic career,NaN,No,North or Central America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,Lack of funding opportunities in my home country,NaN,NaN,Higher salaries post-study,NaN,Family reasons,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,5th,NaN,NaN,2nd,...,NaN,Worked out an individualized development plan,Discussed my career future with a supervisor/PI,NaN,NaN,NaN,NaN,LinkedIn,Twitter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, please specify",tried to change small things that happened on ...,NaN,25 - 34,Male (including trans male),NaN,NaN,NaN,NaN,NaN,South Asian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Yes,Yes,"Yes, I would like Nature Careers to contact me...","Yes, I would like to join the Shift Learning r...",2,NaN
2,SNAP 2015 SHIFT,completed,English,NaN,06/14/2019,17:15:21,06/14/2019,17:39:13,23.87,Doctorate degree (PhD/DPhil/MD),NaN,NaN,Personal interest in my subject of choice,NaN,Yes,North or Central America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Worked out an individualized development plan,Discussed my career future with a supervisor/PI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Nothing,NaN,NaN,I wish I had attacked my deficit in coding and...,25 - 34,Male (including trans male),Caucasian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Yes,Yes,NaN,NaN,3,NaN
3,SNAP 2015 SHIFT,completed,English,NaN,06/14/2019,16:51:28,06/14/2019,17:46:58,55.50,Doctorate degree (PhD/DPhil/MD),NaN,NaN,I want to pursue an academic career,NaN,No,North or Central America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,NaN,NaN,NaN,Lack of quality PhD programmes in my home country,NaN,NaN,Chance to pursue a specific research question,NaN,NaN,NaN,To experience another culture,NaN,NaN,NaN,No,NaN,NaN,1st,NaN,2nd,NaN,NaN,NaN,3rd,...,Developed my social media profile,NaN,NaN,NaN,NaN,NaN,NaN,LinkedIn,NaN,NaN,WeChat,Blogs/Online threads,NaN,NaN,NaN,Change area of study,NaN,NaN,NaN,NaN,NaN,NaN,25 - 34,Male (including trans male),NaN,NaN,NaN,NaN,NaN,NaN,East Asian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Yes,Yes,"Yes, I would like Nature Careers to contact me...","Yes, I would like to join the Shift Learning r...",4,

## Deal with One-Hot Encodings
Many question responses are one-hot encoded. We check that they are all in fact check boxes (cf. radio boxes)

### The Colon Questions

In [8]:
# Create a dictionary of "colon questions" with key the question
#  and val the subquestion
subqs   = lfilter(lambda q : ':' in q, questions.index)
colonQs = itertools.groupby(subqs, key = lambda q : q.split(':')[0]) # Returns [(group, list_of_elts)]
colonQs = {group[0] : list(group[1]) for group in colonQs}

In [9]:
def count_q_responses(row):
    return pd.Series({
              q : (~row[qs].isna()).sum()
              for q, qs in colonQs.items()
        })

Timer.start()
counts = df.swifter.apply(count_q_responses, axis = 'columns')
Timer.end()

/Users/rcharan/anaconda3/envs/learn37/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



109.792 seconds elapsed


In [10]:
# Here are the counts:
pd.concat((counts[q].value_counts() for q in colonQs), axis = 'columns').fillna(0)

,Q12,Q32,Q35,Q39,Q45,Q46,Q47,Q48,Q49,Q52,Q53,Q54,Q58,Q59
0,4353.0,5352.0,5422.0,5522.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,691.0,941.0,900.0,174.0,2856.0,1942.0,766.0,1296.0,556.0,1208.0,3643.0,6306.0,6536.0,6701.0
2,378.0,376.0,347.0,172.0,1256.0,907.0,862.0,1071.0,905.0,1573.0,2098.0,484.0,242.0,99.0
3,531.0,112.0,98.0,316.0,1263.0,1154.0,5184.0,4445.0,5351.0,1802.0,824.0,22.0,31.0,11.0
4,403.0,30.0,37.0,301.0,789.0,1068.0,0.0,0.0,0.0,1110.0,194.0,0.0,3.0,0.0
5,259.0,1.0,7.0,200.0,383.0,755.0,0.0,0.0,0.0,625.0,50.0,0.0,0.0,0.0
6,126.0,0.0,1.0,83.0,164.0,474.0,0.0,0.0,0.0,328.0,3.0,0.0,0.0,0.0
7,45.0,0.0,0.0,32.0,60.0,270.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,0.0
8,19.0,0.0,0.0,10.0,26.0,134.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
9,5.0,0.0,0.0,2.0,14.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Answers are in fact unique
uniqueness_dict = {q : len(df[q].value_counts()) for qs in colonQs.values() for q in qs}
for val in uniqueness_dict.values():
    assert val == 1

In [12]:
# Record the question meanings before 1-hot encoding
subq_dict = {q : df[q].value_counts().index[0] for qs in colonQs.values() for q in qs}

In [13]:
for q in subq_dict:
    df[q] = ~df[q].isna()

### Some other 1-hot encodings

In [14]:
for q in df.columns:
    l = len(df[q].value_counts())
    if l == 0:
        drop_col(df, q)
        print(f'Dropping column {q} because it is empty')
    if l == 1:
        print(f'{q} has only one non-missing unique value')
        print(f'{q}: {questions.loc[q].iloc[0]}')

ID.completed has only one non-missing unique value
ID.completed: Case completed in Snap Interviewer
Dropping column Q1.a because it is empty
Dropping column Q2 because it is empty
Dropping column Q7.a because it is empty
Dropping column Q8.a because it is empty
Dropping column Q9.a because it is empty
Dropping column Q10.a because it is empty
Q11.a has only one non-missing unique value
Q11.a: If other, please specify
Q63 has only one non-missing unique value
Q63: Springer Nature is keen to update PhD students with advic...
Q64 has only one non-missing unique value
Q64: Shift Learning carry out paid research in the education s...
Dropping column Q65.b because it is empty


In [15]:
drop_col(df, 'Q11.a', 'Q63', 'Q64')

In [16]:
df['ID.completed'].value_counts(dropna = False)

completed    6561
NaN           251
Name: ID.completed, dtype: int64

In [17]:
df['ID.completed'] = ~df['ID.completed'].isna()

### Dot Questions

In [18]:
subqs2  = lfilter(lambda q : re.match('Q\d+\.[a-z]', q), df.columns)

In [19]:
dotQs   = itertools.groupby(subqs2, key = lambda q : q.split('.')[0]) # Returns [(group, list_of_elts)]
dotQs   = {group[0] : list(group[1]) for group in dotQs}

In [20]:
soloDotQs = [q for q, qs in dotQs.items() if len(qs) == 1]
for q in soloDotQs:
    print(f'Question {q} has only one sub question')
    if q in questions.index:
        print(f'{q}: {questions.loc[q].iloc[0]}')
    for qs in dotQs[q]:
        l = len(df[qs].value_counts())
        print(f'{qs}: {questions.loc[qs].iloc[0]} --- '
              f'Total unique answer values: {l}')
        if l > 10:
            print(f'*** Dropping column {qs} ***')
            drop_col(df, qs)
    print('---------------\n')

Question Q3 has only one sub question
Q3: Which was the most important reason you decided to enrol in a PhD programme?
Q3.a: If other, please specify --- Total unique answer values: 163
*** Dropping column Q3.a ***
---------------

Question Q6 has only one sub question
Q6: Which region in Asia?
Q6.a: If other, please specify --- Total unique answer values: 5
---------------

Question Q12 has only one sub question
Q12.a: If other, please specify --- Total unique answer values: 152
*** Dropping column Q12.a ***
---------------

Question Q14 has only one sub question
Q14: What is your main reason for having a job?
Q14.a: If other, please specify --- Total unique answer values: 190
*** Dropping column Q14.a ***
---------------

Question Q17 has only one sub question
Q17: Overall, what do you enjoy most about life as a PhD student?       
Q17.a: If other, please specify --- Total unique answer values: 210
*** Dropping column Q17.a ***
---------------

Question Q18 has only one sub question


In [21]:
df.head()

,ID.format,ID.completed,ID.language,ID.site,ID.date,ID.start,ID.endDate,ID.end,ID.time,Q1,Q3,Q4,Q5,Q6,Q6.a,Q7,Q8,Q9,Q10,Q11,Q12:1,Q12:2,Q12:3,Q12:4,Q12:5,Q12:6,Q12:7,Q12:8,Q12:9,Q12:10,Q12:11,Q13,Q14,Q15.a,Q15.b,Q15.c,Q15.d,Q15.e,Q15.f,Q15.g,Q15.h,Q15.i,Q15.j,Q15.k,Q15.l,Q15.m,Q15.n,Q16,Q17,Q18.a,...,Q50.j,Q50.k,Q50.l,Q50.m,Q51.a,Q51.b,Q51.c,Q52:1,Q52:2,Q52:3,Q52:4,Q52:5,Q52:6,Q52:7,Q52:8,Q53:1,Q53:2,Q53:3,Q53:4,Q53:5,Q53:6,Q53:7,Q54:1,Q54:2,Q54:3,Q54:4,Q54:5,Q55,Q56,Q57,Q58:1,Q58:2,Q58:3,Q58:4,Q58:5,Q58:6,Q58:7,Q58:8,Q58:9,Q58:10,Q58:11,Q58:12,Q59:1,Q59:2,Q59:3,Q59:4,Q59:5,Q60,Q61,Q62
0,SNAP 2015 SHIFT,True,Spanish,NaN,06/14/2019,16:46:41,06/14/2019,17:36:11,49.50,Doctorate degree (PhD/DPhil/MD),No job I want is available without a PhD,Yes,North or Central America,NaN,NaN,NaN,NaN,NaN,Mexico,NaN,False,False,False,False,False,False,False,False,False,False,False,No,NaN,NaN,1st,NaN,NaN,7th,3rd,4th,NaN,NaN,5th,NaN,2nd,6th,NaN,La presión por pertenecer a la comunidad cient...,University/academic environment,Somewhat satisfied,...,Badly,Neither well nor badly,Very badly,Very badly,Agree,Disagree,Agree,True,True,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,True,La actitud positiva y constructiva no lo es to...,25 - 34,Female (including trans female),False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,Felicidades por esta encuesta. Pienso que se h...,Yes,Yes
1,SNAP 2015 SHIFT,True,English,NaN,06/14/2019,17:10:02,06/14/2019,17:38:25,28.38,Doctorate degree (PhD/DPhil/MD),I want to pursue an academic career,No,North or Central America,NaN,NaN,NaN,NaN,NaN,United States,NaN,False,False,True,False,False,True,False,True,False,False,False,No,NaN,NaN,NaN,NaN,5th,NaN,NaN,2nd,1st,NaN,NaN,3rd,NaN,4th,NaN,The commercialization of scientific research a...,Intellectual challenge,Somewhat satisfied,...,Badly,Badly,Badly,Badly,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,True,False,False,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,NaN,25 - 34,Male (including trans male),False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,NaN,Yes,Yes
2,SNAP 2015 SHIFT,True,English,NaN,06/14/2019,17:15:21,06/14/2019,17:39:13,23.87,Doctorate degree (PhD/DPhil/MD),Personal interest in my subject of choice,Yes,North or Central America,NaN,NaN,NaN,NaN,NaN,United States,NaN,False,False,False,False,False,False,False,False,False,False,False,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I dont think the grant/award/phd system recogn...,Intellectual challenge,Very satisfied,...,Well,Neither well nor badly,Badly,Badly,Strongly agree,Agree,Agree,False,True,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,I wish I had attacked my deficit in coding and...,25 - 34,Male (including trans male),True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,NaN,Yes,Yes
3,SNAP 2015 SHIFT,True,English,NaN,06/14/2019,16:51:28,06/14/2019,17:46:58,55.50,Doctorate degree (PhD/DPhil/MD),I want to pursue an academic career,No,North or Central America,NaN,NaN,NaN,NaN,NaN,United States,NaN,False,True,False,False,True,False,False,False,True,False,False,No,NaN,1st,NaN,2nd,NaN,NaN,NaN,3rd,NaN,NaN,4th,NaN,NaN,NaN,NaN,The trade war between China and USA.,University/academic environment,Somewhat satisfied,...,Well,Badly,Well,Badly,Agree,Agree,Agree,True,True,True,False,False,False,False,False,True,False,False,True,True,False,False,True,False,False,False,False,NaN,25 - 34,Male (including trans male),False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,NaN,Yes,Yes
4,SNAP 2015 SHIFT,True,English,NaN,06/14/2019,17:39:18,06/14/2019,17:54:03,14.75,Doctorate degree (PhD/DPhil/MD),No job I want is available without a PhD,Ye

### Other Free-form responses

In [22]:
# From ID.endDate and up are to be dropped.
#  7 was an initial cutoff because of the Likert Scale (1-7) commonly used in surveys
unique_counts = df.apply(lambda col : len(col.value_counts())).sort_values(ascending = False)
unique_counts[unique_counts > 7]

ID.start      6499
ID.end        6425
Q26           4920
Q41           4312
Q55           4195
ID.time       2949
Q16           2758
Q60           1080
ID.date         38
ID.endDate      38
Q9              36
Q6              32
Q8              19
ID.site         18
Q40             15
Q15.e           14
Q15.a           14
Q15.b           14
Q15.c           14
Q15.d           14
Q15.f           14
Q15.n           14
Q15.g           14
Q15.h           14
Q15.i           14
Q15.j           14
Q15.k           14
Q15.l           14
Q15.m           14
Q10             12
Q11             11
Q17             10
Q24              9
dtype: int64

In [23]:
# This has recoverable data (coding the smallest countries to 'Other')
df['Q9'].value_counts()

Germany                       528
United Kingdom                526
Czech Republic                181
Switzerland                   155
France                        145
Spain                         140
Netherlands                   132
Italy                         105
Belgium                        71
Portugal                       62
Austria                        59
Sweden                         45
Ireland                        45
Norway                         42
Poland                         34
Denmark                        33
Finland                        30
Greece                         24
Turkey                         23
Russia                         19
Croatia                        17
Hungary                        11
Slovenia                       10
Serbia and Montenegro           9
Romania                         8
Estonia                         6
Slovakia (Slovak Republic)      6
Luxembourg                      5
Lithuania                       3
Cyprus        

In [24]:
drop_col(df, *unique_counts[unique_counts > 37].index.tolist())

### Mass Handling of Variables

In [25]:
small_categorical = {}
boolean           = {}
large_categorical = {}
ordinal           = {}
check_box         = {}

def any_match(pattern, list_):
    return any(map(
        lambda elt : bool(re.match(pattern, elt)),
        list_
    ))

def is_ordinal(values_list):
    try:
        return any_match(
           '''\d\d?\D|.*hour|.*[sS]omewhat|.*Strongly|.*[vV]ery''',
            values_list
        )
    except:
        return False

for col in df.columns:
    values_list = lmap(str, df[col].value_counts(dropna = False).index.tolist())
    values_list.sort()
    
    if is_ordinal(values_list):
        ordinal[col] = values_list
    
    elif col in subqs:
        check_box[col] = values_list
    
    elif len(values_list) == 2:
        boolean[col] = values_list
        
    elif len(values_list) <= 16:
        small_categorical[col] = values_list
        
    else:
        large_categorical[col] = values_list

In [26]:
class_dict =  {
    'ORDINAL'   : ordinal,
    'BOOLEAN'   : boolean,
    'SMALL CAT' : small_categorical,
    'CHECK BOX' : check_box,
    'LARGE CAT' : large_categorical
}

verbose = False

# To review the classifications, set verbose to True and obtain a nice printout
if verbose:
    for name, fields in class_dict.items():
        for col, vals in fields.items():
            qtitle = str(questions.loc[col].iloc[0])
            print(f'{name} | Classified column {col}: {qtitle[:40]}...\n'
                  f'           {str(vals)[:80]}...')
        print('\n'*5)

### Handle Ordinal Data

In [27]:
def recode_ordinal(val):
    val = str(val)
    
    if val == 'nan':
        return np.nan
    
    # Look for simple ordinal scales
    num = re.match('''(\d\d?)(?:st|nd|rd|th|\s=|$)''', val)
    if num:
        return int(num.group(1))
    
    # Look for ranges
    range_ = re.match('''(\d\d?)\s?(?:-|to)\s?(\d\d?)''', val)
    if range_:
        range_ = range_.groups()
        return (int(range_[0]) + int(range_[1])) // 2
    
    if re.search('''([Uu]nsure)|([Oo]ther)|([Pp]refer.*not.*say)|([dD]on't know)''', val):
        return np.nan
    
    # Look for strongly agree/disagree scales
    agreement_dict = {
        'agree'    : '''(A|\sa)gree|[Mm]ore|\ssatisfied|[wW]ell''',
        'disagree' : '''[Dd]isagree|[Ll]ess|dissatisfied|[bB]adly''',
        'somewhat' : '''^\w+$|[Ss]omewhat''',
        'strongly' : '''[Ss]trongly|[Mm]uch|[vV]ery''',
        'equal'    : '''[eE]qual''',
        'alt_1'    : '''[Nn]ot.*at all''',
        'likely'   : '''[Ll]ikely''',
        'unlikely' : '''[Uu]nlikely''',
        'not likely': '''[Nn]ot.*likely'''
    }
    
    for k, pattern in agreement_dict.items():
        agreement_dict[k] = bool(re.search(pattern, val))
        
    custom_dict = {
        'Less than 11 hours'          : 5,
        'More than 80 hours'          : 85,
        'Between one and three hours' : 2, 
        'Less than an hour'           : 1,
        'More than three hours'       : 3,
        'Less than a year'            : 0,
        'More than 6 years'           : 6,
        '65 or over'                  : 69
    }
    
    if val in custom_dict:
        return custom_dict[val]
    

        
    if agreement_dict['agree'] or agreement_dict['disagree']:
        if agreement_dict['agree'] and agreement_dict['disagree']:
            return 3
        elif agreement_dict['agree']:
            added_score = 1
        elif agreement_dict['disagree']:
            added_score = -1
        else:
            assert False
        
        if agreement_dict['somewhat']:
            return 3 + added_score
        elif agreement_dict['strongly']:
            return 3 + added_score * 2
        else:
            assert False
        
    if agreement_dict['equal']:
        return 3
    if agreement_dict['alt_1']:
        return 1
    
    # Handle a weird edge case
    if val == 'Somewhat':
        return 3
    elif val == 'Barely':
        return 2
    elif val == 'Substantially':
        return 4
    elif val == 'Dramatically':
        return 5
    
    # And one more
    if agreement_dict['likely'] or agreement_dict['unlikely'] or agreement_dict['not likely']:
        if agreement_dict['likely'] and agreement_dict['unlikely']:
            return 3
        if agreement_dict['not likely']:
            return 2
        if agreement_dict['likely'] and agreement_dict['strongly']:
            return 5
        if agreement_dict['likely']:
            return 4

In [28]:
# To see the results, uncomment
# for x in more_itertools.unique_everseen(ordinal.values()):
#     dict_ = dict(zip(x, lmap(recode_ordinal, x)))
# #     if any(map(pd.isnull, dict_.values())):
# #         print(dict_)
#     print(pd.Series(dict_).to_string())

In [29]:
# For testing
# ordinal_vals = itertools.chain.from_iterable(ordinal.values())
# ordinal_vals = list(more_itertools.unique_everseen(ordinal_vals))

In [30]:
Timer.start()
for col in ordinal:
    df[col] = df[col].map(recode_ordinal)
Timer.end()

6.971 seconds elapsed


In [31]:
len(df)

6812

### Handle the countries/large categories

In [32]:
questions.loc['Q5':'Q11.a']

,Question Text
Q5,Where do you currently live?
Q6,Which region in Asia?
Q6.a,"If other, please specify"
Q7,Which country in Australasia?
Q7.a,"If other, please specify"
Q8,Which country in Africa?
Q8.a,"If other, please specify"
Q9,Which country in Europe?
Q9.a,"If other, please specify"
Q10,Which country in North or Central America?


In [33]:
region_col   = 'Q5'
country_cols = questions.loc['Q6':'Q11.a'].index.tolist()
country_cols = lfilter(lambda col : 'a' not in col, country_cols)

In [34]:
# Verify that the data is structured as expected:
#  One column has the region, another has a country.
#  Every row should have both
def count_country_responses(row):
    count = (~row[country_cols].isna()).sum()
    return pd.Series({'count' : count})

counts = df.apply(count_country_responses, axis = 'columns')
counts['count'].value_counts()

1    6812
Name: count, dtype: int64

In [35]:
df[region_col].value_counts(dropna = False)

Europe                          2487
North or Central America        1866
Asia (including Middle East)    1857
South America                    223
Australasia                      214
Africa                           165
Name: Q5, dtype: int64

In [36]:
def agg_country_responses(row):
    country = row[country_cols].dropna().iloc[0]
    return pd.Series({'country' : country})

In [37]:
df = df.assign(country = df.apply(agg_country_responses, axis = 'columns'))

In [38]:
# We will cut off the countries coded above 5% of the sample.
#  All others will be coded back to Other - Region
country_counts = df.country.value_counts()
top_countries  = country_counts[country_counts > len(df)*0.05]
top_countries

United States     1548
China              765
India              528
Germany            528
United Kingdom     526
Name: country, dtype: int64

In [39]:
def recode_countries(row):
    if row['country'] in top_countries:
        country = row['country']
    else:
        country = f'''Other - {row['Q5']}'''
    
    return pd.Series({'country' : country})

In [40]:
df = df.assign(country = df.apply(recode_countries, axis = 'columns'))

In [41]:
df.country.value_counts()

United States                           1548
Other - Europe                          1433
China                                    765
Other - Asia (including Middle East)     564
Germany                                  528
India                                    528
United Kingdom                           526
Other - North or Central America         318
Other - South America                    223
Other - Australasia                      214
Other - Africa                           165
Name: country, dtype: int64

In [42]:
country_cols = questions.loc['Q6':'Q11.a'].index.tolist()
country_cols = filter(lambda col : col in df.columns, country_cols)

In [43]:
drop_col(df, 'Q5', *country_cols)

### Make a codebook

In [44]:
class_dict.keys()

dict_keys(['ORDINAL', 'BOOLEAN', 'SMALL CAT', 'CHECK BOX', 'LARGE CAT'])

In [45]:
questions

,Question Text
ID.format,The published format which was employed
ID.completed,Case completed in Snap Interviewer
ID.language,What language would you like to complete the s...
ID.site,Questionnaire location
ID.date,Date of interview
...,...
Q62,Nature may want to contact you again to ask fo...
Q63,Springer Nature is keen to update PhD students...
Q64,Shift Learning carry out paid research in the ...
Q65.a,Name:


In [46]:
question = questions.assign(var_type = np.nan)

In [47]:
# Mark Categorical Variables
categorical_cols = list(itertools.chain(small_categorical.keys(), large_categorical.keys()))
questions.loc[categorical_cols,'var_type'] = 'Categorical'

In [48]:
# Mark Checkbox Variables
questions.loc[check_box.keys(), 'var_type'] = 'Check Box'

In [49]:
questions.loc[boolean.keys(), 'var_type'] = 'Boolean'

In [50]:
questions.loc[ordinal.keys(), 'var_type'] = 'Ordinal'

In [51]:
#Miscellaneous Cleanup
drop_col(df, 'Q61', 'Q62') 

In [52]:
drop_by_rule(questions, questions.apply(lambda row : row.name not in df.columns, axis = 'columns'))

In [53]:
country_var = pd.Series({
    'Question Text' : 'Where do you currently live',
    'var_type'      : 'Categorical'
}, name = 'country')

In [54]:
questions = questions.append(country_var)

In [55]:
# Add the question text for checkboxes
subqs = pd.Series(subq_dict, name = 'checkbox_text')
questions = pd.concat([questions, subqs], axis = 'columns', sort = True)

In [56]:
questions = questions.assign(notes = np.nan)

In [57]:
# Add notes to the Codebook
def add_note(pattern, note_text):
    rows = lfilter(lambda row : re.match(pattern, row),
                  questions.index)
    questions.loc[rows, 'notes'] = note_text

# Based on the original codebook
add_note('Q12:', 'Only for students studying outside their home country (Q4)')
add_note('Q14' , 'Only for students having an extra job (Q13)')
add_note('Q15' , 'What concerns you the most since you started your PhD (ranking)')
add_note('Q19' , 'How satisfied are you with each of the following attributes or aspects of your PhD?')
add_note('Q27' , 'Agreement with statement (from strong disagree to strong agree)')
add_note('Q29' , 'Only for students answering yes to seeking help for anxiety/depression (Q28)')
add_note('Q30' , 'Asked to all; agreement with statement')
add_note('Q32:', 'Only for students answering yes to experiencing bullying (Q31)')
add_note('Q33' , 'Only for students answering yes to experiencing bullying (Q31)')
add_note('Q35:', 'Only for students answering yes to experiencing discrimination or harassment (Q34)')
add_note('Q36' , 'How much do you expect your PhD to improve your job prospects?')
add_note('Q37' , 'Which of the following sectors would you most like to work in (beyond a postdoc) when you complete your degree? (Ranking)')
add_note('Q38' , 'Likelihood of pursuing career path')
add_note('Q39' , 'Only for students unlikely (1-2 out of 5) to pursue Academic Research (Q38)')
add_note('Q40' , 'Only for students likely (4-5 out of 5) to pursue Research (Q38)')
add_note('Q44' , 'Only for students more likely (4-5 out of 5) to pursue Research compared to starting PhD (Q43)')
add_note('Q50' , 'How well is your programme preparing you to carry out each of the following activities?')
add_note('Q51' , 'Agreement with statement (from strong disagree to strong agree)')
add_note('Q57' , 'Gender Identity')

In [58]:
# Add the number of categories and number of missing values

In [59]:
def get_variable_data(var_name):
    out = {}
    out['num_categories'] = len(df[var_name].value_counts(dropna = False))
    out['num_missing']    = df[var_name].isna().sum()
    return pd.Series(out, name = var_name)

In [60]:
to_add = pd.concat([get_variable_data(var_name) for var_name in questions.index], axis = 'columns').transpose()
questions = pd.concat([questions, to_add], sort = True, axis = 'columns')

In [61]:
questions.to_csv('../codebook.csv')

In [62]:
df.to_csv('../dataset.csv')